Подведите результаты эксперимента в экселе по следующим данным: ab_stats.csv - Google Диск:

Стат значимо ли отличается ARPPU в двух группах? Какие рекомендации дадите менеджеру?

In [1]:
# Импортируем библиотеки
import pandas as pd

In [2]:
data = pd.read_csv('ab_stats.csv')
data.head()

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
